# Project 1: Higgs Boson Detection

In [3]:
import numpy as np
import csv
from regression_tools import * 
from cross_validation_ridge import *
from preprocessing import *
import matplotlib.pyplot as plt

## Loading files

In [4]:
def load_data(train_path,test_path):
    train_reader=np.genfromtxt(train_path,delimiter=',',skip_header=1,converters={1:lambda s: float(0) if s==b'b' else float(1)})
    y_train=train_reader[:,1]
    x_train=train_reader[:,2:]
    test_reader=np.genfromtxt(test_path,delimiter=',',skip_header=1)
    x_test=test_reader[:,2:]
    ids_test=test_reader[:,0]
    return x_train,y_train,x_test,ids_test

In [ ]:
#print(phi_train.shape)

In [5]:
#test di durata soluzione analitica vs SGD preparazione dati
x_train,y_train,x_test,ids_test=load_data('train.csv','test.csv')
x_train_cleaned,aa=cleaning_function(x_train)
x_train_cleaned,aaa=features_augmentation(x_train_cleaned,not_augm_features=aa+1)

In [8]:
phi_train=build_polinomial(x_train_cleaned,degree=12,not_poly_features=aa+aaa+1)
phi_train=norm_data(phi_train,skip_first_col=True,not_norm_features=aa+1)
lambda_=10**-3

In [7]:
def norm_data(x,skip_first_col=False,not_norm_features=0):
    if skip_first_col:
        beg=1
    else:
        beg=0
    x_to_norm=x[:,beg:x.shape[1]-not_norm_features]
    means=np.mean(x_to_norm,axis=0)
    sds=np.std(x_to_norm,axis=0)
    x_to_norm=(x_to_norm-means.reshape(1,-1))/sds.reshape(1,-1)
    x[:,beg:x.shape[1]-not_norm_features]=x_to_norm
    return x

In [ ]:
# logistic regression
loss,w=logistic_regression_gradient_descent_demo(y_train, phi_train,max_iter = 10000,
    threshold = 1e-8,gamma=10**-9)

In [ ]:
#test di durata soluzione analitica vs SGD: calcolo soluzione analitica
import time

start = time.time()
loss,w=ridge_regression(y_train,phi_train,lambda_)
end = time.time()
print(end - start)

In [ ]:
#test di durata soluzione analitica vs SGD: calcolo SGD
initial_w=np.zeros(len(w))
batch_size=1
max_iters=100
gamma=0.01
start=time.time()
loss_r, w_r =ridge_regression_SGD(y_train,phi_train,lambda_,initial_w, batch_size, max_iters, gamma)
end=time.time()
print(end-start)

In [ ]:
print(w_r,w)

In [ ]:
#x_train,nmc_tr=cleaning_function(x_train,-999)
#x_test,nmc_te=cleaning_function(x_test,-999)
# nmc= nb of not measured column, i.e. columns where 
#it is present at least one not measured element
#phi_train=build_polinomial(x_train,4,nmc_tr)

In [ ]:
def build_k_indices(y, k_fold, seed):
    """build k indices for k-fold."""
    num_row = y.shape[0]
    interval = int(num_row / k_fold)
    np.random.seed(seed)
    indices = np.random.permutation(num_row)
    k_indices = [indices[k * interval: (k + 1) * interval]
                 for k in range(k_fold)]
    return np.array(k_indices)

In [ ]:
def logistic_cross_validation(y, phi, k_indices, k, lambda_, degree, not_poly_features):
    """return the loss of ridge/linear regression."""
    """Probabilmente conviene implementare una per la regressione normale, in modo da capire 
    quale sia il grado massimo oltre il quale non ha senso andare e poi lavorare con lambda 
    per capire come eliminare feature"""
    
    
        
    train_indices = np.delete(k_indices , k , 0).reshape((k_indices.shape[0]-1) * k_indices.shape[1])
    x_test = phi[k_indices[k],:]
    x_train = phi[train_indices,:]
    y_test = y[k_indices[k]]
    y_train = y[train_indices]
    
    
    tx_train = build_polinomial(x_train, degree, not_poly_features)
    tx_test = build_polinomial(x_test, degree, not_poly_features)
    
    if lambda_!=0:
        loss , w = ridge_regression(y_train, tx_train, lambda_)
    else:
        loss , w = least_squares(y_train,tx_train)
            
    result=(y_test==(tx_test.dot(w)>0.5)).sum()/y_test.shape[0]
    return result

In [ ]:
#def cross_validation_degree():
def cross_validation_logistic_demo(y_train,x_train,degrees,k_fold,lambdas,seed):

    # split data in k fold
    k_indices = build_k_indices(y_train, k_fold, seed)

    # Clean data 
    x_train_cleaned,nmc_tr=cleaning_function(x_train,-999)
    #feature augmentation
    x_train_cleaned,noaf=features_augmentation(x_train_cleaned,not_augm_features=nmc_tr+1)

    # cross validation
    cost_te=np.zeros((lambdas.size,degrees.size))
    for ind_lamb,lambda_ in enumerate(lambdas):
        print(lambda_)
        if lambda_!=0:
            x_train_cleaned=norm_data(x_train_cleaned,not_norm_features=noaf+nmc_tr+1)
        for ind_deg, degree_ in enumerate(degrees):
            #print('DEGREE IS: ')
            #print(degree_)
            loss_te = np.zeros(k_fold)
            for k in range (k_fold):
                #print('K CONSIDERED IS: ')
                #print(k)
                result = LOO_cross_validation(y_train, x_train_cleaned, k_indices, k , lambda_, degree_, nmc_tr+1+noaf)
                loss_te[k]= result

            cost_te[ind_lamb,ind_deg]=loss_te.mean()
    return cost_te


    #cross_validation_degree()

In [ ]:
# Plot the results  
def plot_cross_validation(lambdas,cost_te,degrees):
    plt.figure
    string=[]
    for s in range(lambdas.size):
        plt.plot(degrees,cost_te[s])
        string.append(str(lambdas[s]))
    plt.xlabel('degree')
    plt.ylabel('train accuracy')
    plt.legend(string)
    plt.show()

In [ ]:
def find_the_maximum(matrix):
    max_col=np.max(matrix,axis=0)
    max_col_ind=np.max(np.argmax(max_col))
    max_matrix=np.max(max_col)
    max_row_ind=np.min(np.argmax(matrix[:,max_col_ind]))
    return max_matrix,[max_row_ind,max_col_ind] 

In [ ]:
def create_csv_submission(ids, y_pred, name):
    """
    Creates an output file in csv format for submission to kaggle
    Arguments: ids (event ids associated with each prediction)
               y_pred (predicted class labels)
               name (string name of .csv output file to be created)
    """
    with open(name, 'w') as csvfile:
        fieldnames = ['Id', 'Prediction']
        writer = csv.DictWriter(csvfile, delimiter=",", fieldnames=fieldnames)
        writer.writeheader()
        for r1, r2 in zip(ids, y_pred):
            writer.writerow({'Id':int(r1),'Prediction':int(r2)})

# 81% DO NOT TOUCH

In [ ]:

x_train,y_train,x_test,ids_test=load_data('train.csv','test.csv')
seed = 1
degrees = np.arange(5,16)
k_fold = 4
# To use ridge regression
lambdas = np.logspace(-8,-1,num=5)
print(lambdas.size)
cost_te=cross_validation_demo(y_train,x_train,degrees,k_fold,lambdas,seed)
plot_cross_validation(lambdas,cost_te,degrees)
_,best_param_ind=find_the_maximum(cost_te)
print('best degree is '+str(degrees[best_param_ind[1]]))
print('best lambda is '+str(lambdas[best_param_ind[0]]))


In [ ]:
# continuation of the previous script
x_train_cleaned,nmc_tr=cleaning_function(x_train,-999)
x_train_cleaned=norm_data(x_train_cleaned,not_norm_features=nmc_tr+1)
phi_tr=build_polinomial(x_train_cleaned,degree=degrees[best_param_ind[1]],not_poly_features=nmc_tr+1)
loss,w=ridge_regression(y_train,phi_tr,lambdas[best_param_ind[0]])
x_test_cleaned,nmc_te=cleaning_function(x_test,-999)
x_test_cleaned=norm_data(x_test_cleaned,not_norm_features=nmc_te+1)
phi_te=build_polinomial(x_test_cleaned,degree=degrees[best_param_ind[1]],not_poly_features=nmc_te+1)
y_test=phi_te.dot(w)
y_pred=[]
for i in range(y_test.shape[0]):
    if y_test[i]>0.5:
        y_pred.append(1)
    else:
        y_pred.append(-1)
        #b=-1
        
create_csv_submission(ids_test, y_pred, 'submission.csv')

In [ ]:
print(y_test.shape[0])

In [ ]:
print(ids_test)

In [ ]:
? np.genfromtxt()

In [ ]:
np.nan

In [ ]:
? np.mean

In [ ]:
? np.argmax

In [ ]:
a=np.array([1,2,3,4,5])
b=np.concatenate([a.reshape(-1,1),a.reshape(-1,1)],axis=1)
print(b,a[0:1])

In [ ]:
? np.array

In [ ]:
print(str('b'))

In [2]:
def logistic_regression_gradient_descent_demo(y, tx,gamma,max_iter,threshold):
    # init parameters
    losses = []
    w = np.zeros((tx.shape[1],1))
    # start the logistic regression
    for iter in range(max_iter):
        # get loss and update w.
        loss, w = learning_by_gradient_descent(y.reshape(-1,1), tx, w, gamma)
        # converge criterion
        losses.append(loss)
        if len(losses) > 1 and np.abs(losses[-1] - losses[-2]) < threshold:
            break
    return loss,w

In [ ]:
#logistic_regression_gradient_descent_demo(y, x)

In [ ]:
def logistic_regression_newton_method_demo(y, tx):
    # init parameters
    max_iter = 100
    threshold = 1e-8
    lambda_ = 0.1
    losses = []
    gamma = 1
    w = np.zeros((tx.shape[1],1))
    # start the logistic regression
    for iter in range(max_iter):
        # get loss and update w.
        loss, w = learning_by_newton_method(y.reshape(-1,1), tx, w, gamma)
        # converge criterion
        losses.append(loss)
        if len(losses) > 1 and np.abs(losses[-1] - losses[-2]) < threshold:
            break
    return loss,w

In [ ]:
#logistic_regression_newton_method_demo(y, x)

In [ ]:
def logistic_regression_penalized_gradient_descent_demo(y, tx):
    # init parameters
    max_iter = 10000
    gamma = 0.01
    lambda_ = 0.1
    threshold = 1e-8
    losses = []
    w = np.zeros((tx.shape[1], 1))

    # start the logistic regression
    for iter in range(max_iter):
        # get loss and update w.
        loss, w = learning_by_penalized_gradient(y, tx, w, gamma, lambda_)
        # converge criterion
        losses.append(loss)
        if len(losses) > 1 and np.abs(losses[-1] - losses[-2]) < threshold:
            break
            
    return loss,w

In [ ]:
#logistic_regression_penalized_gradient_descent_demo(y, x)

In [ ]:
x_train,y_train,x_test,ids_test=load_data('train.csv','test.csv')
x_train_cleaned,aa=cleaning_function(x_train)
tx=build_polinomial(x_train_cleaned,degree=1,not_poly_features=0)
tx.shape

In [ ]:
compare_aic(y_train,tx)

In [ ]:
a=list([1,2,3])
a.append(4)
print(a)
a[1]

In [ ]:
a=[]
a.append(0)

temp=a.copy()
temp.append(2)
temp

In [ ]:
def compare_aic(y,tx):
    dimx=tx.shape[1]
    loss=np.zeros(dimx) #contains loss for all models with m variables
    best_loss=np.zeros(dimx) #contains best loss of model with m variables
    models=[] #list of best models
    variables=list(range(dimx)) #list of variables
    for ind in range(dimx):
        for m in variables:
            temp=models.copy()
            #print(m)
            #print(ind)
            temp.append(m)
            #print(temp)
            #print(tx[:,temp].shape)
            [loss[m],w]=logistic_regression_gradient_descent_demo(y,tx[:,temp])
        b=np.argmmin(loss)
        models.append(b)
        variables.remove(b)
        best_loss[ind]=loss.min()
        
    idx_loss=np.argmin(best_loss)
    model=models[:idx_loss]
    return model